In [1]:
import pandas as pd

In [5]:
df=pd.read_csv("data_formed1.csv")

In [6]:
df

,Number of employees Last avail. yr,Cash & Cash Equivalents at the End of Period th USD Last avail. yr,Cash & Cash Equivalents at the End of Period th USD Year - 1,Cash & Cash Equivalents at the End of Period th USD Year - 2,Cash & Cash Equivalents at the End of Period th USD Year - 3,Cash & Cash Equivalents at the End of Period th USD Year - 4,Cash & Cash Equivalents at the End of Period th USD Year - 5,Cash & Cash Equivalents at the Beginning of Period th USD Last avail. yr,Cash & Cash Equivalents at the Beginning of Period th USD Year - 1,Cash & Cash Equivalents at the Beginning of Period th USD Year - 2,...,Ordinary dividends th USD Year - 3.4,Ordinary dividends th USD Year - 4.4,Ordinary dividends th USD Year - 5.4,Dividend share capital other th USD Last avail. yr.4,Dividend share capital other th USD Year - 1.4,Dividend share capital other th USD Year - 2.4,Dividend share capital other th USD Year - 3.4,Dividend share capital other th USD Year - 4.4,Dividend share capital other th USD Year - 5.4,Moody's Long-Term Rating
0,2100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6048000.0,6102000.0,6124000.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
1,70496.0,60279204.0,79887736.0,55261870.0,47388268.0,48840536.0,21664534.0,79887736.0,55261870.0,47388268.0,...,3936000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
2,1541000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
3,385691.0,17046024.0,13398674.0,8650064.0,16331826.0,17387124.0,17909580.0,13744663.0,9248496.0,16019321.0,...,7420108.0,9296486.0,4337823.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
4,62000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14652000.0,13798000.0,13001000.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,2256.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
1080,5000.0,427118.0,354850.0,851012.0,1069214.0,110865.0,116149.0,325161.0,932232.0,1061020.0,...,121722.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
1081,15018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1236900.0,1082300.0,456100.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0
1082,30106.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17300.0,17100.0,16900.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0


In [9]:
merged_df=df.drop("Moody's Long-Term Rating",axis=1)
y=df["Moody's Long-Term Rating"]

In [10]:
learn_x=merged_df[:900].values
learn_y=y[:900].values
test_x=merged_df[900:].values
test_y=y[900:].values

In [11]:
test_x.shape

(184, 3673)

In [12]:
test_y.shape

(184,)

In [16]:
from sklearn.preprocessing import StandardScaler

# DataFrameの全体を正規化
scaler = StandardScaler()
merged_df = pd.DataFrame(scaler.fit_transform(merged_df), columns=merged_df.columns)

In [19]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(merged_df)
n_components = 1000 # 削減後の次元数
pca = PCA(n_components=n_components)
pca_df = pd.DataFrame(pca.fit_transform(merged_df))

In [21]:
learn_x=pca_df[:900].values
learn_y=y[:900].values
test_x=pca_df[900:].values
test_y=y[900:].values

In [22]:
test_x.shape

(184, 1000)

In [23]:

test_y.shape

(184,)

In [24]:
#成型官僚
#tensorflowで実験

In [25]:
import tensorflow as tf

In [26]:
import numpy as np

In [28]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder

# ハイパーパラメータ
k = 5 # k-fold CVの分割数
num_epochs = 15 # エポック数
batch_size = 10 # バッチサイズ

# CVを行うためのインデックスを作成
kf = KFold(n_splits=k, shuffle=True, random_state=1234)

# CVのループ
mae_list = []
for train_index, val_index in kf.split(learn_x):
    # train/valデータを作成
    train_x, train_y = learn_x[train_index], learn_y[train_index]
    val_x, val_y = learn_x[val_index], learn_y[val_index]

    # モデルを構築
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_dim=learn_x.shape[1]),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1),
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    # モデルを学習
    history = model.fit(train_x, train_y, epochs=num_epochs, batch_size=batch_size, validation_data=(val_x, val_y))

    # テストデータで評価
    test_loss, test_mae = model.evaluate(test_x, test_y)
    mae_list.append(test_mae)

# 結果を出力
print('Test MAE:', np.mean(mae_list))

Epoch 1/15
72/72 [==============================] - 2s 13ms/step - loss: 76.8884 - mae: 6.7325 - val_loss: 27.6289 - val_mae: 4.0482
Epoch 2/15
72/72 [==============================] - 0s 4ms/step - loss: 13.5833 - mae: 2.7969 - val_loss: 17.1918 - val_mae: 3.0777
Epoch 3/15
72/72 [==============================] - 0s 4ms/step - loss: 6.2878 - mae: 1.8437 - val_loss: 17.4666 - val_mae: 3.0666
Epoch 4/15
72/72 [==============================] - 0s 4ms/step - loss: 3.3572 - mae: 1.3255 - val_loss: 14.9101 - val_mae: 2.6725
Epoch 5/15
72/72 [==============================] - 0s 4ms/step - loss: 2.3580 - mae: 1.0640 - val_loss: 15.5836 - val_mae: 2.7015
Epoch 6/15
72/72 [==============================] - 0s 4ms/step - loss: 1.8442 - mae: 0.8901 - val_loss: 14.7545 - val_mae: 2.6463
Epoch 7/15
72/72 [==============================] - 0s 4ms/step - loss: 1.7232 - mae: 0.8345 - val_loss: 15.2149 - val_mae: 2.6776
Epoch 8/15
72/72 [==============================] - 0s 4ms/step - loss: 1.7971 -

6/6 [==============================] - 0s 5ms/step - loss: 2.8025 - mae: 1.2296
Epoch 1/15
72/72 [==============================] - 2s 8ms/step - loss: 58.1732 - mae: 6.0741 - val_loss: 24.0485 - val_mae: 3.5182
Epoch 2/15
72/72 [==============================] - 0s 4ms/step - loss: 12.4483 - mae: 2.6020 - val_loss: 21.5093 - val_mae: 3.1807
Epoch 3/15
72/72 [==============================] - 0s 4ms/step - loss: 4.9690 - mae: 1.6773 - val_loss: 20.5975 - val_mae: 2.9989
Epoch 4/15
72/72 [==============================] - 0s 4ms/step - loss: 2.7304 - mae: 1.2007 - val_loss: 19.7573 - val_mae: 2.8950
Epoch 5/15
72/72 [==============================] - 0s 4ms/step - loss: 2.0446 - mae: 1.0026 - val_loss: 20.7820 - val_mae: 2.8658
Epoch 6/15
72/72 [==============================] - 0s 4ms/step - loss: 1.8202 - mae: 0.9277 - val_loss: 20.4019 - val_mae: 2.9563
Epoch 7/15
72/72 [==============================] - 0s 4ms/step - loss: 1.8112 - mae: 0.8977 - val_loss: 20.4819 - val_mae: 2.8345
E

In [274]:
model.save("model")

INFO:tensorflow:Assets written to: model\assets


In [36]:
predictions = model.predict(test_x)
for i in range (len(test_y)):
    print(predictions[i][0].round(),test_y[i])

13.0 14.0
18.0 18.0
15.0 15.0
15.0 16.0
14.0 14.0
14.0 14.0
15.0 15.0
12.0 12.0
13.0 13.0
15.0 14.0
11.0 10.0
15.0 11.0
15.0 13.0
8.0 9.0
14.0 14.0
15.0 16.0
13.0 12.0
15.0 16.0
15.0 14.0
13.0 13.0
13.0 13.0
11.0 11.0
11.0 11.0
11.0 11.0
13.0 13.0
12.0 12.0
12.0 15.0
4.0 2.0
15.0 17.25
15.0 15.0
13.0 12.0
14.0 13.0
13.0 13.0
10.0 9.0
9.0 11.0
10.0 13.0
15.0 15.0
13.0 12.0
13.0 12.0
14.0 14.0
13.0 13.0
11.0 10.0
15.0 14.0
8.0 8.0
13.0 14.0
13.0 13.0
13.0 13.0
15.0 14.0
13.0 14.0
17.0 17.0
11.0 11.0
16.0 16.0
12.0 13.0
14.0 13.0
15.0 15.0
13.0 12.0
11.0 12.0
10.0 15.0
14.0 14.0
14.0 14.0
13.0 14.0
15.0 15.0
16.0 16.0
13.0 14.0
13.0 13.0
14.0 14.0
14.0 15.0
14.0 14.0
16.0 16.0
21.0 21.0
13.0 13.0
13.0 13.0
12.0 13.25
12.0 12.0
11.0 11.0
15.0 17.0
12.0 12.0
14.0 15.0
11.0 13.0
7.0 7.0
12.0 12.0
11.0 12.0
4.0 4.0
14.0 14.0
14.0 12.0
13.0 13.0
13.0 14.0
13.0 13.0
9.0 9.0
6.0 6.0
7.0 7.0
16.0 16.0
3.0 2.0
7.0 7.0
16.0 14.0
21.0 21.0
15.0 14.0
15.0 15.0
14.0 13.0
14.0 14.0
15.0 14.0
15.0 14.0


In [30]:
test_y

array([14.  , 18.  , 15.  , 16.  , 14.  , 14.  , 15.  , 12.  , 13.  ,
       14.  , 10.  , 11.  , 13.  ,  9.  , 14.  , 16.  , 12.  , 16.  ,
       14.  , 13.  , 13.  , 11.  , 11.  , 11.  , 13.  , 12.  , 15.  ,
        2.  , 17.25, 15.  , 12.  , 13.  , 13.  ,  9.  , 11.  , 13.  ,
       15.  , 12.  , 12.  , 14.  , 13.  , 10.  , 14.  ,  8.  , 14.  ,
       13.  , 13.  , 14.  , 14.  , 17.  , 11.  , 16.  , 13.  , 13.  ,
       15.  , 12.  , 12.  , 15.  , 14.  , 14.  , 14.  , 15.  , 16.  ,
       14.  , 13.  , 14.  , 15.  , 14.  , 16.  , 21.  , 13.  , 13.  ,
       13.25, 12.  , 11.  , 17.  , 12.  , 15.  , 13.  ,  7.  , 12.  ,
       12.  ,  4.  , 14.  , 12.  , 13.  , 14.  , 13.  ,  9.  ,  6.  ,
        7.  , 16.  ,  2.  ,  7.  , 14.  , 21.  , 14.  , 15.  , 13.  ,
       14.  , 14.  , 14.  , 13.  , 15.  , 14.  , 14.  , 15.  , 17.  ,
       15.  , 15.  , 12.  , 18.  , 13.  ,  9.  ,  7.  , 14.  , 14.  ,
       13.  ,  9.  , 12.  , 12.  ,  5.  , 10.  , 10.  , 17.  , 10.  ,
       11.  , 13.  ,